In [7]:
import pyvisa
import time
import schedule
import pandas as pd
import serial.tools.list_ports
import os

Port_Arduino = 'COM10'

# Destination folder and file path
folder_path = "C:\\Users\\kary\\Documents\\9_data\\IEAP\\"
file_path = os.path.join(folder_path, "voltage_data.csv")

#Create Resource manager instance
rm = pyvisa.ResourceManager()
#Check the VISA device
rm.list_resources()

('USB0::0x05E6::0x6500::04537333::INSTR', 'ASRL10::INSTR')

In [8]:
# Start the communication by VISA identifying resource name
dmm = rm.open_resource('USB0::0x05E6::0x6500::04537333::INSTR')

#Checking the machine name
print(dmm.query('*IDN?'))

KEITHLEY INSTRUMENTS,MODEL DMM6500,04537333,1.7.11b



In [14]:
dmm.write('*RST')
dmm.write(':DISPlay:VOLTage:DIGits 3')
#dmm.write("SENS:FUNC 'VOLT:DC', (@1)")
#dmm.write("SENS:FUNC 'RES', (@6)")
#dmm.write("SENS:FUNC 'RES', (@7)")

27

### With Position

In [15]:
#列を5つもつデータフレームに新しい行を追加する
def add_row(df,row):
    new_row = pd.Series({df.columns[0]:row[0],df.columns[1]:row[1],df.columns[2]:row[2]}).to_frame().T   
    # new_row = pd.Series({df.columns[0]:row[0],df.columns[1]:row[1],df.columns[2]:row[2],df.columns[3]:row[3],df.columns[4]:row[4],df.columns[5]:row[5]}).to_frame().T
    return  pd.concat([df,new_row],ignore_index = True)

In [16]:
#データフレーム、マルチメータ、合計測定時間、サンプル周期、印可電圧時間
def df_add_read_data(df,mul,total_read_time,sample_period):
    #測定時間の初期化
    start_time =  time.perf_counter()
    end_time = time.perf_counter()
    while end_time - start_time < total_read_time:
        each_start_time =  time.perf_counter()        
        dmm.write("SENS:FUNC 'VOLT:DC', (@6)")
        dmm.write('ROUT:CLOS (@6)')
        data1 = float(mul.query("READ?"))
        dmm.write("SENS:FUNC 'VOLT:DC', (@7)")
        dmm.write('ROUT:CLOS (@7)')
        data2 = float(mul.query("READ?"))

        each_end_time = time.perf_counter()
        #サンプル周期になるまで待機
        while  each_end_time - each_start_time <= sample_period:
            each_end_time = time.perf_counter()
            
        df = add_row(df,[each_end_time - start_time,data1,data2])
        end_time = time.perf_counter()
    return df

#### Measure

In [17]:
columns = ["Time[s]","Measured_Voltage[V]","Resistance[Ohm]"]
df_voltage = pd.DataFrame(columns=columns)

position = 0
read_time = 5 # sec
freq = 0.001
df_voltage = df_add_read_data(df_voltage,dmm,read_time,freq)
df_voltage

C:\Users\kary\AppData\Local\Temp\ipykernel_19252\1442998754.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return  pd.concat([df,new_row],ignore_index = True)


,Time[s],Measured_Voltage[V],Resistance[Ohm]
0,0.249851,9.900000e+37,0.000616
1,0.343283,9.900000e+37,0.004518
2,0.460059,9.900000e+37,0.005067
3,0.552016,9.900000e+37,0.004984
4,0.666066,9.900000e+37,0.004956
5,0.758733,9.900000e+37,0.004959
6,0.852124,9.900000e+37,0.005023
7,0.966224,9.900000e+37,0.005090
8,1.058478,9.900000e+37,0.005176
9,1.172959,9.900000e+37,0.005151
